In [ ]:
import os,sys

code_dir = os.path.expandvars('$OSCBAGDIS_DATAPROC_CODE')
sys.path.append(code_dir)

import mne
import utils  #my code
import json
import matplotlib.pyplot as plt
import numpy as np
import h5py
%load_ext autoreload
%autoreload 2

if os.environ.get('DATA_DUSS') is not None:
    data_dir = os.path.expandvars('$DATA_DUSS')
else:
    data_dir = '/home/demitau/data'
    


#rawname_ = 'S01_off_hold'
#rawname_ = 'S01_off_move'
#rawname_ = 'S01_on_hold'
#rawname_ = 'S01_on_move'

#rawname_ = 'S02_off_hold'
#rawname_ = 'S02_off_move'
#rawname_ = 'S02_on_hold'
#rawname_ = 'S02_on_move'

#rawname_ = 'S03_off_hold'
#rawname_ = 'S03_off_move'

#rawname_ = 'S04_off_hold'
#rawname_ = 'S04_off_move'
#rawname_ = 'S04_on_hold'
#rawname_ = 'S04_on_move'

#rawname_ = 'S05_off_hold'
#rawname_ = 'S05_off_move'
#rawname_ = 'S05_on_hold'
#rawname_ = 'S05_on_move'

# rawname_ = 'S06_off_hold'
# rawname_ = 'S06_off_move'
# rawname_ = 'S06_on_hold'
#rawname_ = 'S06_on_move'

#rawname_ = 'S07_off_hold'
#rawname_ = 'S07_off_move'
#rawname_ = 'S07_on_hold'
#rawname_ = 'S07_on_move'

#rawname_ = 'S08_off_rest'
#rawname_ = 'S08_on_rest'

#rawname_ = 'S09_off_rest'

#rawname_ = 'S10_off_rest'
#rawname_ = 'S10_off_move'

rawname_ = 'S99_off_move'

rawname = rawname_ + '_resample_raw.fif'
fname_full = os.path.join(data_dir,rawname)
    
# read file -- resampled to 256 Hz,  Electa MEG, EMG, LFP, EOG channels
raw = mne.io.read_raw_fif(fname_full, None)
sfreq = raw.info['sfreq']

import utils_preproc as upre
mod_info, infos = upre.readInfo(rawname_, raw)
raw.info = mod_info

anns_fn = rawname_ + '_anns.txt'
anns_fn_full = os.path.join(data_dir, anns_fn)
anns = mne.read_annotations(anns_fn_full)
raw.set_annotations(anns)

# get info about bad MEG channels (from separate file)
with open('subj_info.json') as info_json:
        #raise TypeError

    #json.dumps({'value': numpy.int64(42)}, default=convert)
    gen_subj_info = json.load(info_json)
    
subj,medcond,task  = utils.getParamsFromRawname(rawname_)
subj_num = int(subj[1:])
if subj_num < 8:
    badchlist = gen_subj_info[subj]['bad_channels'][medcond][task]
else:
    badchlist = gen_subj_info[subj]['bad_channels'][medcond]['hold']
raw.info['bads'] = badchlist
print('bad channels are ',badchlist)
for chn in badchlist:
    if chn.find('LFP') >= 0:
        print('WARNING: bad LFP channel: {}'.format(chn))

In [ ]:
mne.__version__

In [ ]:
#raw.annotations

In [ ]:
# import utils
# trem_times_fn = 'trem_times_tau.json'
# with open(trem_times_fn ) as jf:
#     trem_times_byhand = json.load(jf)   
# trem_times_nms_fn = 'trem_times_tau_nms.json'
# with open(trem_times_nms_fn ) as jf:
#     trem_times_nms_byhand = json.load(jf)   

# #%debug
# tremIntervalJan, artif         = utils.unpackTimeIntervals(trem_times_byhand, mainSide = True, 
#                                                            gen_subj_info=gen_subj_info, skipNotLoadedRaws=0)
# tremIntervalJan_nms, artif_nms = utils.unpackTimeIntervals(trem_times_nms_byhand, mainSide = False, 
#                                                            gen_subj_info=gen_subj_info, skipNotLoadedRaws=0)
# for rawn in [rawname_]:
#     if rawn in artif_nms and rawn not in artif:
#         artif[rawn] = artif_nms[rawn]
#     else:
#         if rawn in artif_nms:
#             artif[rawn].update(artif_nms[rawn] )
        
# for rawn in tremIntervalJan:
#     sind_str,medcond,task = utils.getParamsFromRawname(rawn)
#     maintremside = gen_subj_info[sind_str]['tremor_side']
#     opside= utils.getOppositeSideStr(maintremside)
#     if rawn in tremIntervalJan_nms:
#         tremIntervalJan[rawn][opside] = tremIntervalJan_nms[rawn][opside] 


# mvtTypes = ['tremor', 'no_tremor', 'unk_activity']

# plotTremNegOffset = 2.
# plotTremPosOffset = 2.
# maxPlotLen = 6   # for those interval that are made for plotting, not touching intervals for stats
# addIntLenStat = 5
# plot_time_end = 150

# timeIntervalPerRaw_processed = utils.processJanIntervals(tremIntervalJan, maxPlotLen, addIntLenStat, 
#                           plotTremNegOffset, plotTremPosOffset, plot_time_end, mvtTypes=mvtTypes)

In [ ]:
# for current raw
maintremside = gen_subj_info[subj]['tremor_side']
mts_letter = maintremside[0].upper()
print(maintremside)

#raw.info['bads'] = []

raw_lfponly = raw.copy()
raw_lfponly.info['bads'] = []
#raw_lfponly.crop(0,300)
raw_lfponly.load_data()

#hand_side = 'L'
#hand_side = 'R'
hand_side = ''

if hand_side == 'L':
    brain_side  = 'R'
elif hand_side == 'R':
    brain_side  = 'L'
else:
    brain_side = ''
chis = mne.pick_channels_regexp(raw.ch_names, 'LFP{}.*'.format(brain_side))
chnames_lfp = [raw.ch_names[chi] for chi in chis]
print(chnames_lfp)

raw_lfponly.pick_channels(   chnames_lfp  )
print(raw_lfponly.ch_names)

print( raw_lfponly.get_channel_types() )

y = {}
for chname in raw_lfponly.ch_names:
    y[chname] = 'eeg'
raw_lfponly.set_channel_types(y)

In [ ]:
supp_info_fname_noext = rawname_ + '_supp_info'
supp_info_fname = supp_info_fname_noext + '.npz'
supp_info_fname_full = os.path.join(data_dir,supp_info_fname)
supp_info = np.load(supp_info_fname_full,allow_pickle=1)['supp_info'][()]
pairs = supp_info['artfiacts_intervals_LFP']
anns_LFPartif = utils.intervals2anns(pairs,int_name='BAD_LFP')

In [ ]:
raws = {}
raws['untouched'] = raw_lfponly

cutoff_freq = 2
rawcur = raw_lfponly.copy()
freqsToKill = np.arange(50, 128, 50) # harmonics of 50
rawcur.notch_filter(freqsToKill, picks=['eeg'])
raws['raw_lfponly_notched'] = rawcur

rawcur = raws['raw_lfponly_notched'].copy()
rawcur.filter(l_freq=cutoff_freq,h_freq=None)
raws['raw_lfponly_notched_filtered'] = rawcur

rawcur = raws['raw_lfponly_notched'].copy()
pad_type = 'symmetric'
rawcur.filter(l_freq=cutoff_freq,h_freq=None,pad=pad_type)
raws['raw_lfponly_notched_filtered_pad={}'.format(pad_type)] = rawcur

rawcur = raws['raw_lfponly_notched'].copy()
rawcur.set_annotations(anns_LFPartif)
rawcur.filter(l_freq=cutoff_freq,h_freq=None,pad=pad_type,
                                         skip_by_annotation='BAD_LFP')
raws['raw_lfponly_notched_filtered_safe_pad={}'.format(pad_type)] = rawcur

#%matplotlib qt

#help(raw_lfponly.plot)

In [ ]:
chn = 'LFPR01'
figsz = (12,3)
for rl in raws:
    r = raws[rl]
    chd,ts = r[chn]
    plt.figure(figsize=figsz)
    plt.plot(ts,chd[0])
    #plt.xlim(5.6,12.5)
    plt.title(rl)
    plt.show()

In [ ]:
chd.shape

In [ ]:
rawbest = raws['raw_lfponly_notched_filtered_safe_pad=symmetric'];

In [ ]:
rawdata = rawbest.get_data()

%matplotlib inline
raw_lfponly.plot_psd(fmin=0,fmax=30);

#%matplotlib inline
raw_lfponly.plot_psd(fmin=30,fmax=sfreq/2);
#raw_lfponly.filter(l_freq=cutoff_freq,h_freq=None)

In [ ]:
help(raw_lfponly.plot_psd)

In [ ]:
rawname_

In [ ]:
for chi in range(len(rawbest.ch_names)):
    chn = rawbest.ch_names[chi]
    #raw_lfponly.plot_psd(fmin=0,fmax=30, picks=chn);
    fig = rawbest.plot_psd(fmin=0, picks=chn, show=False);
    #fig.suptitle(chn)
    fig.gca().set_title(chn)
    plt.plot()

# Save filtered

In [ ]:
rawname_LFPonly = rawname_ + '_LFPonly'+ '.fif'
rawname_LFPonly_full = os.path.join( data_dir, rawname_LFPonly )
rawbest.save(rawname_LFPonly_full,overwrite=1)

In [ ]:
print(raw_lfponly.ch_names)
chdata = raw_lfponly.get_data()

#chdata = chdata[:,:256*20]

#pha.shape

#amp.shape
plt.figure(figsize=(15,5))
ax = plt.gca()
sh = 0
for chi in range(len(chdata) ):
    dd = chdata[chi,:]
    dd = np.diff(dd)
    ax.plot(raw.times[1:],dd + sh, label=raw_lfponly.ch_names[chi])
    sh += ( np.quantile(dd,.95)-np.quantile(dd,.05) ) * 2.3
    
ax.legend(loc='upper right')
ax.set_xlim(raw.times[0],raw.times[-1])
plt.suptitle('Raw diff')

In [ ]:
print(rawbest.ch_names)
chdata = rawbest.get_data()

#chdata = chdata[:,:256*20]

#pha.shape

#amp.shape
plt.figure(figsize=(15,5))
ax = plt.gca()
sh = 0
for chi in range(len(chdata) ):
    dd = chdata[chi,:]
    ax.plot(raw.times,dd + sh, label=rawbest.ch_names[chi])
    sh += ( np.quantile(dd,.95)-np.quantile(dd,.05) ) * 2.3
    
ax.legend(loc='upper right')
ax.set_xlim(raw.times[0],raw.times[-1])
plt.suptitle('Raw')

In [ ]:
raw_lfponly = rawbest

In [ ]:
import globvars as gv
#gv.fbands
fbands_to_use = ['tremor', 'beta', 'gamma'] #, 'HFO']

raw_per_fb = {}
for fbcur in fbands_to_use:
    fb0,fb1 = gv.fbands[fbcur]
    raw_lfponly_cb = raw_lfponly.copy()
    raw_lfponly_cb.filter(l_freq=fb0,h_freq=fb1)
    raw_per_fb[fbcur] = raw_lfponly_cb

# Observe data

In [ ]:
%matplotlib qt
print(maintremside)

y = filter(lambda x: x.startswith('LFP'), badchlist)
list(y)

print(badchlist)

In [ ]:
#%matplotlib qt

In [ ]:
#%matplotlib qt
#raw_lfponly.plot(duration = 50, scalings={'eeg':'auto'} );

In [ ]:
print(anns.description)

onset = []
dur = []
descr = []
for i,ann in enumerate(raw_lfponly.annotations):
    if ann['description'].find('_{}'.format(mts_letter)) >= 0:
        onset.append(ann['onset'])
        dur.append(ann['duration'])
        descr.append(ann['description'])
ann_mainside = mne.Annotations(onset,dur,descr)
print(ann_mainside)

In [ ]:
#maintremside

In [ ]:
#%matplotlib qt
raw_per_fb['beta'].set_annotations( ann_mainside)
#raw_per_fb['beta'].plot(duration = 50);

In [ ]:
#%matplotlib qt
raw_per_fb['gamma'].set_annotations( ann_mainside)
#raw_per_fb['gamma'].plot(duration = 50);

In [ ]:
help(plt.hist)

# Histograms

In [ ]:
rawdata_flt = raw_lfponly.get_data()
rawdata_flt.shape

pctshift = 1
pct = [pctshift, 100-pctshift]
b0 = np.inf; b1 = -np.inf
center = 0
normalize = 0
for ind in range(rawdata_flt.shape[0] ):
    cd = rawdata_flt[ind].copy()
    if center:
        cd -= np.mean(cd)
    if normalize:
        b0_,b1_ = np.percentile(cd, pct)
        cd /= (b1_-b0_)
    b0_,b1_ = np.percentile(cd, pct)
    plt.hist(cd, bins=100, alpha=0.7, 
             label='{}'.format(raw_lfponly.ch_names[ind]), range=(b0_,b1_))
    b0 = min(b0,b0_)
    b1 = max(b1,b1_)
plt.xlim(b0,b1)

plt.legend()

# TFR

In [ ]:
#help(mne.time_frequency.tfr_array_multitaper)

In [ ]:
wsz_sec = 1
wsz_bins = wsz_sec * int(sfreq)

# I want 256 window sz
cf =  wsz_bins/ ( 5/(2*np.pi) * wsz_bins  ) 
print(cf)

#help(raw_lfponly.get_data)

In [ ]:
# strec = 0
# endrec = raw_lfponly.times[-1]
# epdur = endrec
# events_one = mne.make_fixed_length_events(raw_lfponly, start=strec, stop=endrec, duration=epdur)
# epochs_one = mne.Epochs(raw_lfponly,events_one, tmin=0,tmax = epdur, baseline=None)
decim = 8
#tfr_array_morlet
min_freq = 3
freq_step = 2
freqs = np.arange(min_freq,100,freq_step)
#freq2cycles_mult = 0.75
freq2cycles_mult = cf  # 1.2566370614359172
# tfrres = mne.time_frequency.tfr_array_morlet(raw_lfponly.get_data()[None,:], 
#     raw.info['sfreq'], freqs, freqs * freq2cycles_mult, n_jobs=10, decim = decim)
tfrres = mne.time_frequency.tfr_array_multitaper(raw_lfponly.get_data()[None,:], 
        raw.info['sfreq'], freqs, freqs * freq2cycles_mult, n_jobs=10, decim = decim)
tfrres = tfrres[0]

times = raw_lfponly.times[::decim]
print( tfrres.shape )

In [ ]:
%matplotlib inline
baseline = [times[0],times[-1]]
#baseline = [400,600]
nr = len(tfrres)
hh = 3
fig,axs = plt.subplots(nr, 1, figsize=(24,hh*nr))
plt.subplots_adjust(right=0.99,left=0.05,top=0.96,bottom=0.03)
import matplotlib as mpl
baseline_bins = np.where(np.logical_and(times>= baseline[0], times< baseline[1] ) )[0]
for i in range(nr):
    ax = axs[i]
    dat_ = np.abs( tfrres[i] )
    qs = np.percentile(dat_[:,baseline_bins], [5,95])
    norm = mpl.colors.LogNorm(vmin = qs[0], vmax = qs[1])
    ax.pcolormesh(raw.times[::decim], freqs, dat_, norm=norm )
    ax.set_ylabel(raw_lfponly.ch_names[i])
plt.savefig(os.path.join(gv.dir_fig, '{}_LFP_TFR.png'.format(rawname_)) )
import gc; gc.collect()

In [ ]:
print(raw.info['bads'])
print(gen_subj_info[subj].get('bad_lfp', None ) )
print(gen_subj_info[subj]['lfpchan_used_in_paper'] )

In [ ]:
print(rawname_)

## TFR Histograms

In [ ]:
%matplotlib inline
plt.figure()
nshow = 7

nr = len(tfrres)
hh = 3
fig,axs = plt.subplots(nr, 1, figsize=(15,hh*nr), sharex='col')
import matplotlib as mpl
for i in range(nr):
    ax = axs[i]
    #N =  tfres_.shape[0] 
    dat_ = np.abs( tfrres[i] )
    #qs = np.percentile(dat_, [5,95])
    #norm = mpl.colors.LogNorm(vmin = qs[0], vmax = qs[1])
    for ind in range( 0, len(freqs), len(freqs)//nshow ):
        ax.hist( np.abs( dat_[ind] ) , bins=100, alpha=0.7, 
               label='{:.1f}'.format(freqs[ind]))
    ax.set_ylabel(raw_lfponly.ch_names[i])
    ax.legend(loc='upper right')
import gc; gc.collect()    
    
#plt.legend()

# Look if we have something evidently weird in ICA

In [ ]:
#tfrres.shape

#np.abs( tfrres[i].T )[256:-256].shape

#tfrres[i].shape

n_edge_bins_TFR = wsz_bins // decim
print(n_edge_bins_TFR)

## ICA per channel

In [ ]:
from sklearn.decomposition import FastICA

ica_res = []
n_components = 3
for i in range(len(tfrres)):
    Xfull = tfrres[i].T [n_edge_bins_TFR:-n_edge_bins_TFR]  # to avaoid edge artifact due to wavelet computation
    print(i)
    if decim == 1:
        skip = 10
    else:
        skip = 1
    X = np.log( np.abs( Xfull[::skip] ) )
    Xtimes = times[n_edge_bins_TFR:-n_edge_bins_TFR:skip]
    #print(X.shape, Xtimes.shape)
    assert(X.shape[0] == len(Xtimes)),   [X.shape, Xtimes.shape]

    ica = FastICA(n_components=n_components, random_state=0)
    S_ = ica.fit_transform(X)  
    ica_res += [S_]

In [ ]:
#%matplotlib inline
hh=3
nr = len(ica_res)
fig,axs = plt.subplots(nr, 1, figsize=(24,hh*nr))
plt.subplots_adjust(right=0.99,left=0.05,top=0.96,bottom=0.03)
    
for chi in range(len(ica_res)):
    S_ = ica_res[chi]
    sh = 0.07
    #descrs = ['trem']
    #plt.figure(figsize=(15,2))
    ax = axs[chi]
    for i in range(S_.shape[-1]):
        ax.plot(Xtimes, S_[:,i] + i*sh, label=str(i))
        ax.set_ylabel('{} = {}'.format(chi,raw_lfponly.ch_names[chi]) )
        ax.set_xlim(Xtimes[0],Xtimes[-1])
        #for i in ivalis
    ax.legend(loc='upper left')
        
plt.savefig(os.path.join(gv.dir_fig, '{}_LFP_ICA_per_chan.png'.format(rawname_)) )
import gc; gc.collect()

## ICA per brain side

In [ ]:
chns_perside = {}
chis_perside = {}
for side in ['R', 'L']:
    sidelet = side
    chis = mne.pick_channels_regexp(raw_lfponly.ch_names, 'LFP'+sidelet)
    chis_perside[side] = chis
    chns_perside[side] = [raw_lfponly.ch_names[i] for i in chis]
print(chns_perside)

#tfrres[ chis_perside['L'] ].shape
#help(tfrres.reshape)
n_components = 3

ica_res_perside = {}
for side in chns_perside:
    predat =  tfrres[ chis_perside[side] ]
    predat = predat.reshape( (predat.shape[0] * predat.shape[1], predat.shape[2]) )
    Xfull = predat.T [n_edge_bins_TFR:-n_edge_bins_TFR]  # to avaoid edge artifact due to wavelet computation

    if decim == 1:
        skip = 10
    else:
        skip = 1
    X = np.log( np.abs( Xfull[::skip] ) )
    Xtimes = times[n_edge_bins_TFR:-n_edge_bins_TFR:skip]
    print(X.shape, Xtimes.shape)


    ica = FastICA(n_components=n_components, random_state=0)
    S_ = ica.fit_transform(X)  
    ica_res_perside[side] = S_

%matplotlib inline
for side in chns_perside:
    S_ = ica_res_perside[side]
    sh = 0.07
    #descrs = ['trem']
    plt.figure(figsize=(15,3))
    ax = plt.gca()
    for i in range(S_.shape[-1]):
        ax.plot(Xtimes, S_[:,i] + i*sh, label=str(i))
        ax.set_ylabel(side)
        ax.set_xlim(Xtimes[0],Xtimes[-1])
        #for i in ivalis
    ax.legend(loc='upper left')

import gc; gc.collect()

In [ ]:
print( S_.shape )

#%matplotlib qt
plt.close('all')

In [ ]:
print( rawname_ )

In [ ]:
# wsz_bins = 256
# decim = 8
#print( width_conv_bins )

In [ ]:
# control detection of thr crossing for name something artifact (for the component)
mults = [2.5 ] * len(ica_res)  
# controls where the mean it taken from when centering the component
baselines = [[200,250]] * len(ica_res)
# controls 
width_conv_sec = 0.5
width_conv_bins = int( wsz_bins / decim * width_conv_sec )
print('width_conv_bins = ',width_conv_bins)
thr = 2 / width_conv_bins; 
#extFactorLs = [-0.3] * len(ica_res); extFactorRs = [-0.1] * len(ica_res)
extFactorLs = [0.1] * len(ica_res); extFactorRs = [0.1] * len(ica_res)
percentthr = 0.9
ind_component_to_use_per_channel = [[0]] * len(ica_res)
if rawname_ == 'S02_off_hold':
    baselines = [[0,250]] * len(ica_res)
    mults[1] = 2.2
    extFactorLs[1] = 0
    extFactorRs[1] = 0
    ind_component_to_use_per_channel[3] = [2]
    ind_component_to_use_per_channel[4] = [2]
if rawname_ == 'S02_off_move':
    baselines = [[200,400]] * len(ica_res)
    ind_component_to_use_per_channel[1] = [1]
    ind_component_to_use_per_channel[3] = [2]
    ind_component_to_use_per_channel[4] = [1]
if rawname_ == 'S02_on_hold':
    baselines = [[700,times[-1]]] * len(ica_res)
    ind_component_to_use_per_channel[0] = [1]
    ind_component_to_use_per_channel[1] = [1]
    ind_component_to_use_per_channel[3] = [2]
if rawname_ == 'S01_on_move':
    mults = [3 ] * len(mults)
    baselines = [[0,100]] * len(ica_res)  # first you plot with some values then set it
    #thr = 0.05
    #width_conv_bins=100
if rawname_ == 'S03_off_hold':
    baselines = [[400,800]] * len(ica_res)  # first you plot with some values then set it
if rawname_ == 'S03_off_move':
    baselines = [[100,550]] * len(ica_res) 
    ind_component_to_use_per_channel[3] = [1]
    ind_component_to_use_per_channel[4] = [1]
    ind_component_to_use_per_channel[5] = [0,2]
if rawname_ == 'S04_off_hold':
    baselines = [[320,470]] * len(ica_res) 
    mults[0] = 3
    mults[4] = 4
    ind_component_to_use_per_channel[0] = [1]
    ind_component_to_use_per_channel[3] = [1]
    ind_component_to_use_per_channel[4] = [1]
    ind_component_to_use_per_channel[5] = [2]
if rawname_ == 'S04_off_move':
    baselines = [[110,210]] * len(ica_res)
    mults[3] = 3
    mults[4] = 3.5
    ind_component_to_use_per_channel[4] = [0,2]
    #ind_component_to_use_per_channel[4] = [2]
    ind_component_to_use_per_channel[5] = [2]
if rawname_ == 'S04_on_hold':
    baselines = [[500,600]] * len(ica_res)
    mults[1] = 2
    ind_component_to_use_per_channel[1] = [1]
    ind_component_to_use_per_channel[3] = [1]
    ind_component_to_use_per_channel[4] = [1,2]
    ind_component_to_use_per_channel[5] = [2]
if rawname_ == 'S04_on_move':
    baselines = [[600,700]] * len(ica_res)
    mults[3] = 1.8
    mults[4] = 1.8
    ind_component_to_use_per_channel[0] = [1]
    ind_component_to_use_per_channel[3] = [2]
    ind_component_to_use_per_channel[4] = [2]
if rawname_ == 'S05_on_hold':
    ind_component_to_use_per_channel[0] = [2]
    ind_component_to_use_per_channel[1] = [2]
    ind_component_to_use_per_channel[2] = [2]
    ind_component_to_use_per_channel[3] = [2]
    ind_component_to_use_per_channel[5] = [2]
if rawname_ == 'S05_on_move':
    ind_component_to_use_per_channel[0] = [2]
    ind_component_to_use_per_channel[1] = [2]
    ind_component_to_use_per_channel[2] = [2]
    ind_component_to_use_per_channel[3] = [2]
    ind_component_to_use_per_channel[4] = [2]
    extFactorRs[:] = [0.2] * len(ica_res)
if rawname_ == 'S07_off_hold':
    ind_component_to_use_per_channel[4] = [1]
    ind_component_to_use_per_channel[5] = [2]
if rawname_ == 'S08_on_rest':
    ind_component_to_use_per_channel[8] = [1]
    ind_component_to_use_per_channel[9] = [2]
    ind_component_to_use_per_channel[11] = [2]
    ind_component_to_use_per_channel[12] = [2]
    ind_component_to_use_per_channel[13] = [2]
if rawname_ == 'S09_off_rest':
    baselines = [[300,550]] * len(ica_res)
    ind_component_to_use_per_channel[0] = [2]
    ind_component_to_use_per_channel[1] = [1]
    ind_component_to_use_per_channel[2] = [2]
    ind_component_to_use_per_channel[4] = [1]
    ind_component_to_use_per_channel[5] = [1]
    ind_component_to_use_per_channel[10] = [1]
    ind_component_to_use_per_channel[11] = [1]
    ind_component_to_use_per_channel[12] = [1]
    ind_component_to_use_per_channel[13] = [1]
if rawname_ == 'S10_off_rest':
    baselines = [[60,150]] * len(ica_res)
    ind_component_to_use_per_channel[4] = [2]
    ind_component_to_use_per_channel[5] = [1]
if rawname_ == 'S10_off_move':
    baselines = [[80,200]] * len(ica_res)
    ind_component_to_use_per_channel[0] = [2]
    ind_component_to_use_per_channel[2] = [1]
    mults[2] = 2.2
    

In [ ]:
#assert len(raw.annotations) == 0
#thr = 0.01
import utils_tSNE as utsne
cvls = {}
ivals_check_artifs = {}
anns_check_artifs = {}
dat_absed = {}
artifacts_found = False
plot_only_channels_with_artifacts = 0 
fig,axs = plt.subplots(2*len(ica_res),1,figsize=(24,len(ica_res)*4), sharex='col')
plt.subplots_adjust(right=0.99,left=0.05,top=0.96,bottom=0.03)

for chi in range(len(ica_res)):
    chn = raw_lfponly.ch_names[chi]
    anns_check_artifs[chn] = mne.Annotations([],[],[])
for chi in range(len(ica_res)):
    chn = raw_lfponly.ch_names[chi]
    S_ = ica_res[chi]
    #N_components_to_use = 1
    ival_baseline = baselines[chi]
    b0,b1 = ival_baseline
    inds_baseline = np.where(np.logical_and( (Xtimes >= b0) , (Xtimes <= b1)) )[0]
    for ind_component_to_use in ind_component_to_use_per_channel[chi]:
        dat_cur = S_[:,ind_component_to_use ]
        dat_cur = np.abs( dat_cur - np.mean(dat_cur[inds_baseline]) )
        dat_absed[chn] = dat_cur

        # pctshift = 25
        # pcts = [pctshift, 100-pctshift]
        # qs = np.percentile(megdat, pcts, axis=1)
        #me, mn,mx = utsne.robustMean(checkdat, axis=1, per_dim =1, ret_aux=1, q = .25)
        #checkdat_scaled = ( checkdat - me[:,None] ) / (mx-mn)[:,None]
        #checkdat_sum = np.sum(np.abs(checkdat_scaled),axis=0)
        #dat_cur = S_[:,0]
        me_s, mn_s,mx_s = utsne.robustMean(dat_cur[inds_baseline], 
                                           axis=None, per_dim =1, ret_aux=1, pos = 1)
        mult = mults[chi]
        mask= dat_cur > mx_s * mult
        summask = np.sum(mask)
        if summask != 0:
            #np.where(mask)[0]
            cvl,ivals_check_artif = utils.getIntervals(mask ,include_short_spikes=1,\
                minlen=2, thr=thr, inc=1, extFactorL=extFactorLs[chi], 
                                                       extFactorR=extFactorRs[chi],
                percentthr=percentthr, width=width_conv_bins, min_dist_between=5)

            print('artifact intervals found ' ,ivals_check_artif)
            if chn in cvls:
                cvls[chn] = np.maximum(cvl,cvls[chn] )
            else:
                cvls[chn] = cvl
            ivals_check_artifs[chn] = ivals_check_artif
            if len(ivals_check_artif) > 0:
                artifacts_found = True
        else:
            print('{}, comp {}: no artifacts found'.format(chn,ind_component_to_use))
            ivals_check_artif = []
            if plot_only_channels_with_artifacts:
                continue

        ax = axs[chi * 2]
        ax.plot(Xtimes,dat_cur / mx_s, label='comp {}'.format(ind_component_to_use))
        ax.axhline( me_s / mx_s, c='r', ls=':')
        ax.axhline( mx_s / mx_s, c='purple', ls=':')
        ax.axhline( me_s * mult / mx_s, c='r', ls=':')
        ax.axhline( mx_s * mult / mx_s, c='purple', ls=':')
        ax.set_ylabel(raw_lfponly.ch_names[chi])
        ax.legend(loc='lower right')

        if summask > 0:
            ax = axs[chi * 2 + 1]
            ax.plot(Xtimes,cvl, label='comp {}'.format(ind_component_to_use))
            ax.axhline(y=thr, ls=':', c='purple')
            ax.legend(loc='lower right')
            ax.set_xlim(Xtimes[0],Xtimes[-1])

        for i in [chi * 2, chi * 2 + 1]:
            ax = axs[i]
            if i % 2 == 0:
                ymin = np.min(dat_cur) /  mx_s; ymax = np.max(dat_cur) / mx_s
            else:
                if summask > 0:
                    ymin = 0; ymax = np.max(cvl)
            for ivl in ivals_check_artif:
                b0,b1 = ivl
                b0t,b1t = Xtimes[b0],Xtimes[b1]
                ax.axvline( b0t , c='r', ls=':')
                ax.axvline( b1t , c='r', ls=':')

                ax.fill_between( [b0t,b1t], ymin,ymax, facecolor='red', alpha=0.2)
            ax.set_xlim(Xtimes[0],Xtimes[-1])
            #raw.annotations.append([b0],[b1-b0], ['bad_MEG'])

        anns_check_artifs[chn] +=\
        utils.intervals2anns(ivals_check_artif,\
                             'BAD_{}'.format(chn),
                             Xtimes)



plt.savefig(os.path.join(gv.dir_fig,'{}_LFP_artif_from_ICA.png'.format(rawname_)) )

    #plt.close()
print('artifacts_found =',artifacts_found)

In [ ]:
#artifacts_found = False

In [ ]:
# saving emtpy
if not artifacts_found:
    mne.Annotations([],[],[]).\
    save(os.path.join(gv.data_dir, '{}_ann_LFPartif.txt'.format(rawname_) ) )
    print('Saving empty')

In [ ]:
%matplotlib qt

In [ ]:
#ymin,ymax = ax.get_ylim()

## Plot sub_bands and components togehter (nothing gets saved)

In [ ]:
#%matplotlib inline
#%matplotlib qt
plot_detailed = 0
if plot_detailed:
    for chi in range(len(ica_res)):
        chn = raw_lfponly.ch_names[chi]
        if chn not in cvls:
            continue

        #S_ = ica_res[chi]
        dat_cur = dat_absed[chn]

        #descrs = ['trem']
        plt.figure(figsize=(40,8))
        ax = plt.gca()

        logabsdat = np.log( np.abs(tfrres) )

        pctsh = 5
        mlt = 1.3
        mlt2 = 155
        mlt3 = 455

        effmn, effmx = np.percentile(logabsdat,[pctsh,100-pctsh]) 
        effrange = (effmx - effmn)
        sh = effrange * mlt

        effmn_S, effmx_S = np.percentile(dat_cur,[pctsh,100-pctsh]) 
        effrange = effmx_S - effmn_S


        cvl = cvls[chn]
        ivals_check_artif = ivals_check_artifs[chn]
        cvlrange = np.max(cvl)
        i = 0
        #while i < S_.shape[-1]:
            #ax.plot(Xtimes, S_[:,i] + (i)*sh)
            #for i in ivalis


        ax.plot(Xtimes, (dat_cur - effmn_S) / (effmx_S-effmn_S) * effrange * mlt2 + (i)*sh); i+= 1
        ax.plot(Xtimes,cvl / cvlrange * effrange *mlt3 + (i)*sh)
        ax.set_xlim(raw.times[0],raw.times[-1])

        while i < tfrres.shape[1]:
            ax.plot(times, logabsdat[chi,i,:] + (- effmn  + (i)*sh ) )
            i += 1

        ymin,ymax = ax.get_ylim()
        for ivl in ivals_check_artif:
            b0,b1 = ivl
            b0t,b1t = Xtimes[b0],Xtimes[b1]
            ax.axvline( b0t , c='r', ls=':')
            ax.axvline( b1t , c='r', ls=':')

            ax.fill_between( [b0t,b1t], ymin,ymax, facecolor='red', alpha=0.2)


        plt.tight_layout()
        plt.savefig('{}_LFP_artif_from_ICA_{}.png'.format(rawname_,chn), dpi=300)
        plt.close()
    import gc; gc.collect()

## Extend ends/beginning

In [ ]:
extend_interval_sec = 2
for chn in anns_check_artifs:
    anns = anns_check_artifs[chn]
    newanns = mne.Annotations([],[],[])
    for an in anns:
        if an['onset'] < extend_interval_sec:
            an['onset'] = 0
            print(chn,' extend')
        if an['onset'] + an['duration'] > raw.times[-1] - extend_interval_sec:
            an['duration'] = raw.times[-1] - an['onset']
        newanns.append(an['onset'],an['duration'],an['description'])
    anns_check_artifs[chn] = newanns
    #utils.mergeIntervalsWithinList(newann_curside, printLog=True)

In [ ]:
print(anns_check_artifs)#, anns_check_artifs.onset

## Only select artifacts from some channels

In [ ]:
artifact_approve_chns = raw_lfponly.ch_names # default
artifact_approve_side = ''
#artifact_approve_side = 'R' # if '', then both
if len(artifact_approve_side) > 0:
    print('Filtering')
    artifact_approve_chns = [chn for chn in raw_lfponly.ch_names \
                             if chn.find('LFP{}'.format(artifact_approve_side)) >= 0 ]
print(artifact_approve_chns)

## Only select artifacts from some time periods

In [ ]:
#%debug
#artiact_approve_intervals = [ [0,300] ]
artiact_approve_intervals = [ [0,raw.times[-1]] ]  #entire
anns_check_artifs_upd = {}
for approve_int in artiact_approve_intervals:
    ai0,ai1 = approve_int
    for chi in range(len(ica_res)):
        chn = raw_lfponly.ch_names[chi]
        if chn not in artifact_approve_chns:
            print('skipping {}'.format(chn))
            continue
        if chn not in anns_check_artifs:
            print('skipping {}'.format(chn))
            continue
        anns = anns_check_artifs[chn]
        newanns = utils.findIntersectingAnns(ai0,ai1,anns, ret_type='anns')
        anns_check_artifs_upd[chn] = newanns
        
#Believe artifacts found in time period
print(anns_check_artifs_upd)

In [ ]:
#print( anns_check_artifs['LFPL23'].description )

### Plot

In [ ]:
anns_dict = anns_check_artifs_upd
#anns_dict = anns_check_artifs
for chn in anns_dict:
    plt.figure(figsize=(13,3))
    anns = anns_dict[chn]
    ax = plt.gca()
    for an in anns:
        b0,b1 = an['onset'], an['onset']+an['duration']
        ax.axvline( b0 , c='r', ls=':')
        ax.axvline( b1 , c='r', ls=':')
        ymin = 0; ymax = 1
        ax.fill_between( [b0,b1], ymin,ymax, facecolor='red', alpha=0.2)
    ax.set_title(chn)
    ax.set_xlim(raw.times[0],raw.times[-1])

## Merge artfiacts from all channels on one side (or not)

In [ ]:
#%debug
#merge_by_side = 1
merge_sides = []
#merge_sides = ['L', 'R']
#merge_sides = ['L']
if len(merge_sides):
    newann_perside = {}
    for side in merge_sides:
        newann_curside = mne.Annotations([],[],[])
        templ = 'LFP'+side
        for chn in anns_check_artifs_upd:
            if chn.find(templ) < 0:  # if not this side
                continue
            anns = anns_check_artifs_upd[chn]
            print(chn, np.sum(anns.duration))
            descr = len(anns.duration) * ['BAD_'+templ]
            newann_curside += mne.Annotations(anns.onset,anns.duration,descr)
        newann_curside_merged = utils.mergeIntervalsWithinList(newann_curside, printLog=True)
        for ii in range(30):
        #newann_curside_merged = utils.mergeIntervalsWithinList(newann_curside_merged, printLog=True)
            #print(ii)
            newann_curside_merged = utils.mergeIntervalsWithinList(newann_curside_merged, printLog=True)
        newann_perside[side ] = newann_curside_merged
        
        print(side, np.sum(newann_curside.duration), np.sum(newann_curside_merged.duration)  )
        
        

    for side in newann_perside:
        print(side, newann_perside[side], newann_perside[side].onset)

    #anns_check_artifs_upd['LFPL12' ].description
    #utils.intervals2anns( utils.anns2intervals(anns_check_artifs_upd['LFPL12' ], tuple_len=3 ) )

    #%matplotlib inline
    for side in newann_perside:
        plt.figure()
        anns = newann_perside[side]
        ax = plt.gca()
        for an in anns:
            b0,b1 = an['onset'], an['onset']+an['duration']
            ax.axvline( b0 , c='r', ls=':')
            ax.axvline( b1 , c='r', ls=':')
            ymin = 0; ymax = 1
            ax.fill_between( [b0,b1], ymin,ymax, facecolor='red', alpha=0.2)
        ax.set_title(side)
        ax.set_xlim(raw.times[0],raw.times[-1])

    for side in newann_perside:
        print(side,newann_perside[side].onset)
        print(side,newann_perside[side].duration, np.sum(newann_perside[side].duration))

In [ ]:
ch_names = raw_lfponly.ch_names
info = mne.create_info(ch_names, sfreq/decim, ch_types='eeg')
#print(info)
#help(mne.io.RawArray)

dat = []
for chi in range(len(ica_res)):
    chn = raw_lfponly.ch_names[chi]
    dat += [dat_absed[chn]]
dat = np.vstack(dat)

print(chns_perside)

artif_raw = mne.io.RawArray(dat,info) #,first_samp=n_edge_bins_TFR)
#artif_raw.times += wsz_sec
#Tune by hand
ta = mne.Annotations([],[],[])
for side in merge_sides:
    ta  += newann_perside[side] 
    #print(ta.onset)
if len(merge_sides) < 2:
    nonmerge_sides = list( set(['L','R']) - set(merge_sides) )
    print('nonmerge_sides = ',nonmerge_sides)
    for side in nonmerge_sides:
        for chn in anns_check_artifs_upd:
            if chn in chns_perside[side]:
                ta += anns_check_artifs_upd[chn]
                #print(chn,'fd')
import copy
ta_shifted = copy.deepcopy(ta)
ta_shifted.onset -= wsz_sec
    
artif_raw.set_annotations(ta_shifted)
print(ta_shifted)

#--------------------------

artif_raw_upsampled = artif_raw.copy().resample(256)
print(artif_raw_upsampled.times)
#help(artif_raw_upsampled.get_data)
dat = artif_raw_upsampled.get_data(reject_by_annotation=None)
print(dat.shape)
d1s = np.zeros( (dat.shape[0], wsz_bins) )

tremfreq_for_preproc = 10
rectconvraw = upre.extractEMGData(raw, rawname_ = rawname_, tremfreq=tremfreq_for_preproc)
emgdat = rectconvraw.get_data()

dat_ext = np.hstack( [d1s, dat, d1s] )
assert  dat_ext.shape == rawdata.shape
print(rawdata.shape)
from sklearn.preprocessing import RobustScaler
dd = []
chns = []
for chi in range(len(raw_lfponly.ch_names)):
    chn = raw_lfponly.ch_names[chi]
    dd += [rawdata[chi]]; chns += [chn]
    dd += [dat_ext[chi]]; chns += [chn+'_I0']
dd += [emgdat]
chns += rectconvraw.ch_names
    
dd = np.vstack(dd)
scaler = RobustScaler(quantile_range=(15,85), with_centering=1)
scaler.fit(dd.T)
dd_scaled = scaler.transform(dd.T).T
    
info = mne.create_info(chns, sfreq, ch_types='eeg')
lfp_aug = mne.io.RawArray(dd_scaled,info)
#lfp_aug.set_annotations(raw_lfponly_wann.annotations)
lfp_aug.set_annotations(ta)

In [ ]:
%matplotlib qt

In [ ]:
lfp_aug.plot(duration=50,
                       scalings={'eeg':'auto'}, highpass=None, lowpass=None  );

In [ ]:
print(lfp_aug.annotations)
print(lfp_aug.annotations.onset)

## I allow to have labels that mark bad segemnets in all channel per side. I want to convert them to per-channel labels

In [ ]:
newann = mne.Annotations([],[],[])
for an in lfp_aug.annotations:
    processed = False
    for s in ['LFPL', 'LFPR']:
        d = an['description']
        if not d.endswith( s ):  # not just find, but completely
            continue
        print('found merged per side')
        for chn in raw_lfponly.ch_names:
            print(d,chn)
            if chn.find(s) < 0:
                continue
            newann.append(an['onset'],an['duration'],'BAD_' + chn)
            processed = True
    if not processed:
        newann.append(an['onset'],an['duration'],an['description'])
            
print(newann)
print(newann.onset)
print(newann.description)

In [ ]:
#help(artif_raw.plot)

# %matplotlib qt

# artif_raw.plot(duration=50,scalings={'eeg':'auto'}, highpass=None, lowpass=None);

# print(artif_raw.annotations.onset, artif_raw.annotations.duration)

# raw_lfponly_wann = raw_lfponly.copy()
# taa = artif_raw.annotations
# taa.onset += wsz_sec
# raw_lfponly_wann.set_annotations(taa)

# raw_lfponly_wann.plot(duration=50,scalings='auto', highpass=None, lowpass=None);

# raw_lfponly_wann.annotations



In [ ]:
help(raw_lfponly.plot_psd)

In [ ]:
# Check that after rejecting artifacts 

In [ ]:
raw_lfponly_wann = raw_lfponly.copy()
raw_lfponly_wann.set_annotations(newann)
for chi in range(len(raw_lfponly.ch_names)):
    chn = raw_lfponly.ch_names[chi]
    #raw_lfponly.plot_psd(fmin=0,fmax=30, picks=chn);
    raw_lfponly_wann.plot_psd(fmin=0, picks=chn, reject_by_annotation=True);
    plt.title(chn)

# Save

In [ ]:
#lfp_aug.annotations.\
resname = os.path.join(gv.data_dir, '{}_ann_LFPartif.txt'.format(rawname_) ) 
print(resname)
newann.save(resname)

## Just to look how components are related to labels

In [ ]:
lfp_aug.set_annotations(newann)

In [ ]:
lfp_aug_all_ann = lfp_aug.copy()
lfp_aug_all_ann.set_annotations(raw_lfponly.annotations + lfp_aug.annotations)
lfp_aug_perside = utils.getLFPperSide(lfp_aug_all_ann)

In [ ]:
lfp_aug_perside['L'].plot(duration=50,
                       scalings='auto', highpass=None, lowpass=None  );

In [ ]:
lfp_aug_perside['R'].plot(duration=50,
                       scalings='auto', highpass=None, lowpass=None  );

In [ ]:
# artif_raw_tremann = lfp_aug.copy()
# ann_mainside_sh = ann_mainside
# #ann_mainside_sh.onset -= wsz_sec
# artif_raw_tremann.set_annotations( ann_mainside_sh + raw_lfponly_wann.annotations)

# artif_raw_tremann.plot(duration=50,
#                        scalings='auto', highpass=None, lowpass=None  )

In [ ]:
#artif_raw_tremann.crop

# try PAC

In [ ]:
import tensorpac

sfreq = 256

# define an ERPAC object
p = tensorpac.EventRelatedPac(f_pha=[3,10], f_amp=(30, 90, 5, 5))

# extract phases and amplitudes
pha = p.filter(sfreq, chdata[0], ftype='phase',     n_jobs=6)
amp = p.filter(sfreq, chdata[1], ftype='amplitude', n_jobs=6)

#Compute the ERPAC using the two implemented methods and plot it

# implemented ERPAC methods
#methods = ['circular', 'gc']
methods = [ 'gc']

plt.figure(figsize=(16, 8))
for n_m, m in enumerate(methods):
    # compute the erpac
    erpac = p.fit(pha, amp, method=m, smooth=50, n_jobs=-1).squeeze()

    # plot
    plt.subplot(len(methods), 1, n_m + 1)
    p.pacplot(erpac, time, p.yvec, xlabel='Time (second)' * n_m,
              cmap='Spectral_r', ylabel='Amplitude frequency', title=p.method,
              cblabel='ERPAC', vmin=0., rmaxis=True)
    plt.axvline(1., linestyle='--', color='w', linewidth=2)

p.show()

